In [5]:
import pandas as pd
import numpy as np

# I. Load the data, clean it and extract usefull information

In [21]:
nodes = pd.read_csv("./NTDS_Data/census_and_vote_data.csv" )
migflow2015_county_nodeid_dist = pd.read_csv("./NTDS_Data/migflow2015_county_nodeid_dist.csv")

In [44]:
# drop useless columns : ori_county and des_county use FSPE code as indexes, we are using a inhouse indexing
mig_flow = migflow2015_county_nodeid_dist.drop(columns=["ori_county", "des_county", "dist_km"])

In [50]:
num_nodes = nodes.values.shape[0]
num_flows = mig_flow.shape[0]
print(num_flows)

42409


# II. Create the Adjency matrix

In [49]:
A_total = np.zeros((num_nodes, num_nodes))
A_returns = np.zeros((num_nodes, num_nodes))
A_exemptions = np.zeros((num_nodes, num_nodes))
for i in range (0, num_flows) : 
    A_total[mig_flow["ori_id"] - 1, mig_flow["des_id"] -1] = mig_flow["returns"] + mig_flow["exemptions"]
    A_returns[mig_flow["ori_id"] - 1, mig_flow["des_id"] -1] = mig_flow["returns"]
    A_exemptions[mig_flow["ori_id"] - 1, mig_flow["des_id"] -1] = mig_flow["exemptions"]